In [ ]:
import pynwb
from pathlib import Path
import pandas as pd
import glob
from spike_utils import get_good_units_with_structure, maxInterval, get_burst_times_dict, count_bursts_and_spikes_after_stim, add_metadata_to_df
from allensdk.brain_observatory.ecephys.dynamic_gating_ecephys_session import DynamicGatingEcephysSession


In [ ]:

# Define base path
base_path = Path('G:/SWDB_COLLAB')

# Define all sessions
sessions_info = [
    "607660_20220607",
    "607660_20220609",
    "626279_20220928",
    "633232_20221110",
    "638387_20221201",
    "615048_20220812",
    "615048_20220810",
    "607660_20220608",
    "626279_20220926",
    "633232_20221108"
]

# Define burst parameters (same as before)
burst_params = {
    'max_begin_ISI': 0.004,  # 4 ms
    'max_end_ISI': 0.02,     # 20 ms
    'min_IBI': 0.1,          # 100 ms
    'min_burst_duration': 0.008,  # 8 ms
    'min_spikes_in_burst': 3,
    'pre_burst_silence': 0.1  # 100 ms
}

# Initialize an empty list to store all DataFrames
all_counts_df = []

# Loop through each session
for session_info in sessions_info:
    try:
        # Parse subject_id and session id
        subject_id = f"sub-{session_info.split('_')[0]}"
        session_id = session_info.split('_')[1]

        # Construct session filename pattern
        session_pattern = f"{subject_id}_ses-{session_id}*.nwb"

        # Construct full path to NWB file using glob
        nwb_path_pattern = str(base_path / 'data' / subject_id / session_pattern)
        matching_files = glob.glob(nwb_path_pattern)
        
        # Construct session filename
        session = matching_files[0]
        
        # Construct full path to NWB file
        nwb_path = base_path / 'data' / subject_id / session
        
        print(f"Processing {subject_id} - {session}")
        
        # Load the NWB file
        with pynwb.NWBHDF5IO(str(nwb_path), mode='r', load_namespaces=True) as nwb_file_asset:
            nwb_file = nwb_file_asset.read()
            dynamic_gating_session = DynamicGatingEcephysSession.from_nwb(nwb_file)
            
            # Get stimulus presentations
            stim_presentations = dynamic_gating_session.stimulus_presentations
            flashes = stim_presentations[stim_presentations['stimulus_name'].str.contains('flash')]
            presentation_times = flashes.start_time.values
            
            # Get good units with structure info
            good_units_visual, spike_times_dict, structure_dict = get_good_units_with_structure(dynamic_gating_session)
            
            # Get burst times
            burst_times_dict = get_burst_times_dict(spike_times_dict, burst_params)
            
            # Get structure-wise counts
            counts_df = count_bursts_and_spikes_after_stim(
                burst_times_dict,
                spike_times_dict,
                structure_dict,
                presentation_times,
                window_duration=1.0
            )
            
            # Add metadata
            counts_df = add_metadata_to_df(
                counts_df,
                dynamic_gating_session.metadata,
                session=session,
                subject_id=subject_id
            )
            
            # Append to list of all DataFrames
            all_counts_df.append(counts_df)
            
    except Exception as e:
        print(f"Error processing {session_info}: {str(e)}")
        continue

# Combine all DataFrames
final_df = pd.concat(all_counts_df, ignore_index=True)

# Save the results
output_path = base_path / 'analysis_results'
output_path.mkdir(exist_ok=True)
final_df.to_csv(output_path / 'all_sessions_burst_analysis.csv', index=False)

print("Analysis complete!")
print(f"Total sessions processed: {len(all_counts_df)} out of {len(sessions_info)}")
print(f"Total rows in final DataFrame: {len(final_df)}")